# Домашнее задание № 11. Машинный перевод


Для данных en-de.

# Транформеры для решения seq2seq задач

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

Данные взяты вот отсюда - https://opus.nlpl.eu/opus-100.php (раздел с отдельными языковыми парами)

In [3]:
en_sents = open('/kaggle/input/opus-tr/opus.de-en-train.en').read().splitlines()
de_sents = open('/kaggle/input/opus-tr/opus.de-en-train.de').read().splitlines()

Пример перевода с английского на немецкий

In [4]:
en_sents[-1], de_sents[-1]

('I am not talking about criminalising it or reacting to it emotionally, but we must tackle this subject.',
 'Es geht nicht darum, hier zu kriminalisieren, zu emotionalisieren, sondern wir müssen uns mit dem Thema auseinandersetzen.')

Как обычно нам нужен токенизатор, а точнее даже 2, т.к. у нас два корпуса

In [5]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=["/kaggle/input/opus-tr/opus.de-en-train.en"], trainer=trainer_en)

tokenizer_de = Tokenizer(BPE())
tokenizer_de.pre_tokenizer = Whitespace()
trainer_de = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_de.train(files=["/kaggle/input/opus-tr/opus.de-en-train.de"], trainer=trainer_de)

In [6]:
tokenizer_en.save('/kaggle/working/tokenizer_en')
tokenizer_de.save('/kaggle/working/tokenizer_de')

In [7]:
tokenizer_en = Tokenizer.from_file("/kaggle/working/tokenizer_en")
tokenizer_de = Tokenizer.from_file("/kaggle/working/tokenizer_de")

In [8]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [9]:
# важно следить чтобы индекс паддинга совпадал в токенизаторе с value в pad_sequences
PAD_IDX = tokenizer_de.token_to_id('[PAD]')
PAD_IDX

3

In [10]:
# (разные чтобы показать что в seq2seq не нужна одинаковая длина)
max_len_en, max_len_de = 40, 45

In [11]:
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]
X_de = [encode(t, tokenizer_de, max_len_de) for t in de_sents]

In [12]:
# миллион примеров 
len(X_en), len(X_de)

(1000000, 1000000)

In [13]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_de):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_IDX)
        
        self.texts_de = [torch.LongTensor(sent) for sent in texts_de]
        self.texts_de = torch.nn.utils.rnn.pad_sequence(self.texts_de, padding_value=PAD_IDX)

        self.length = len(texts_en)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[:, index]
        ids_de = self.texts_de[:, index]

        return ids_en, ids_de

трейн и тест

In [14]:
X_en_train, X_de_train = X_en, X_de
en_sents_valid = open('/kaggle/input/opus-tr/opus.de-en-dev.en').read().splitlines()
de_sents_valid = open('/kaggle/input/opus-tr/opus.de-en-dev.de').read().splitlines()
X_en_valid = [encode(t, tokenizer_en, max_len_en) for t in en_sents_valid]
X_de_valid = [encode(t, tokenizer_de, max_len_de) for t in de_sents_valid]

In [15]:
training_set = Dataset(X_en_train, X_de_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True, )

In [16]:
valid_set = Dataset(X_en_valid, X_de_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=200, shuffle=True)

# Код трансформера

Дальше код модели, он взят вот отсюда (с небольшими изменениями) - https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Там есть комментарии по каждому этапу

In [17]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, 
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         print('pos inp')
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         print('pos dec')
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
#         print('pos out')
        x = self.generator(outs)
#         print('gen')
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [18]:
from time import time
def train(model, iterator, optimizer, criterion, print_every=500):
    
    epoch_loss = []
    ac = []
    
    model.train()  

    for i, (texts_en, texts_de) in enumerate(iterator):
        texts_en = texts_en.T.to(DEVICE) # чтобы батч был в конце
        texts_de = texts_de.T.to(DEVICE) # чтобы батч был в конце
        
        # помимо текста в модель еще нужно передать целевую последовательность
        # но не полную а без 1 последнего элемента
        # а на выходе ожидаем, что модель сгенерирует этот недостающий элемент
        texts_de_input = texts_de[:-1, :]
        
        
        # в трансформерах нет циклов как в лстм 
        # каждый элемент связан с каждым через аттеншен
        # чтобы имитировать последовательную обработку
        # и чтобы не считать аттеншн с паддингом 
        # в трансформерах нужно считать много масок
        (texts_en_mask, texts_de_mask, 
        texts_en_padding_mask, texts_de_padding_mask) = create_mask(texts_en, texts_de_input)
        logits = model(texts_en, texts_de_input, texts_en_mask, texts_de_mask,
                       texts_en_padding_mask, texts_de_padding_mask, texts_en_padding_mask)
        optimizer.zero_grad()
        
        # сравниваем выход из модели с целевой последовательностью уже с этим последним элементом
        texts_de_out = texts_de[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_de_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        
    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion):
    
    epoch_loss = []
    epoch_f1 = []
    
    model.eval()  
    with torch.no_grad():
        for i, (texts_en, texts_de) in enumerate(iterator):
            texts_en = texts_en.T.to(DEVICE)
            texts_de = texts_de.T.to(DEVICE)

            texts_de_input = texts_de[:-1, :]

            (texts_en_mask, texts_de_mask, 
            texts_en_padding_mask, texts_de_padding_mask) = create_mask(texts_en, texts_de_input)

            logits = model(texts_en, texts_de_input, texts_en_mask, texts_de_mask,
                           texts_en_padding_mask, texts_de_padding_mask, texts_en_padding_mask)

            
            texts_de_out = texts_de[1:, :]
            loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_de_out.reshape(-1))
            epoch_loss.append(loss.item())
            
    return np.mean(epoch_loss)

In [19]:
torch.manual_seed(0)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
DE_VOCAB_SIZE = tokenizer_de.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, EN_VOCAB_SIZE, DE_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [20]:
# torch.save(transformer, 'model')

In [21]:
# transformer = torch.load('model').to(DEVICE)

In [22]:
torch.cuda.empty_cache()

(эпох мало, т.к. долго учится)

In [23]:
from timeit import default_timer as timer
NUM_EPOCHS = 15

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)
    
    if not losses:
        print(f'First epoch - {val_loss}')
        #torch.save(transformer, 'model')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}')
        #torch.save(transformer, 'model')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

Loss: 7.637119799613953;
Loss: 7.011490802288056;
Loss: 6.66494140847524;
Loss: 6.427875431776047;
Loss: 6.247375998878479;
Loss: 6.100318091392517;
Loss: 5.974966465132577;
Loss: 5.867505427956581;
Loss: 5.771845067871942;
Loss: 5.684390247726441;
First epoch - 4.769833135604858
Epoch: 1, Train loss: 5.684, Val loss: 4.770,            Epoch time=1207.074s
Loss: 4.772817912101746;
Loss: 4.727994191646576;
Loss: 4.688127451578776;
Loss: 4.650930532693863;
Loss: 4.612350955200196;
Loss: 4.575618277549744;
Loss: 4.540472996575492;
Loss: 4.505721473693848;
Loss: 4.470529109319051;
Loss: 4.4378191261768345;
Improved from 4.769833135604858 to 4.0034057855606076
Epoch: 2, Train loss: 4.438, Val loss: 4.003,            Epoch time=1218.737s
Loss: 4.042820899009705;
Loss: 4.013517164945602;
Loss: 3.993954383214315;
Loss: 3.9717069146633146;
Loss: 3.9504026856422425;
Loss: 3.929262145201365;
Loss: 3.907880992957524;
Loss: 3.8874631922245024;
Loss: 3.8682304849624636;
Loss: 3.8488128133773802;
Imp

In [24]:
torch.save(transformer, 'model')

In [25]:
def translate(text):


    input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]
    output_ids = [tokenizer_de.token_to_id('[CLS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_en_mask, texts_de_mask, 
    texts_en_padding_mask, texts_de_padding_mask) = create_mask(input_ids_pad, output_ids_pad)

    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_de_mask,
                   texts_en_padding_mask, texts_de_padding_mask, texts_en_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_de.token_to_id('[SEP]'), tokenizer_de.token_to_id('[PAD]')]:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_en_mask, texts_de_mask, 
        texts_en_padding_mask, texts_de_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_de_mask,
                       texts_en_padding_mask, texts_de_padding_mask, texts_en_padding_mask)
        pred = logits.argmax(2)[-1].item()

    return (' '.join([tokenizer_de.id_to_token(i).replace('##', '') for i in output_ids[1:]]))


In [26]:
translate('Answer a question')

'Antwort eine Frage'

In [27]:
import nltk

In [28]:
en_sents_test = open('/kaggle/input/opus-tr/opus.de-en-test.en').read().lower().splitlines()
de_sents_test = open('/kaggle/input/opus-tr/opus.de-en-test.de').read().lower().splitlines()

In [29]:
translations = []

for i in range(len(en_sents_test)):
      translations.append(translate(en_sents_test[i]))

In [30]:
translations[:4]

['04 : 26 : 35',
 'Vor historischen Arch ä ologie im dr itten reichen reich ".',
 "indem Sie auf ' sa ve profile ', Sie stimmen die Benutzer stimmen , die diese Begriffe und Bedingungen zu .",
 'Ich wollte dir etwas zeigen .']

In [31]:
de_sents_test[:4]

['04:26:35',
 'prähistorische archäologie im dritten reich".',
 'die nutzungsbedingungen werden durch das klicken des nutzers auf "profil speichern" vereinbart.',
 'ich wollte dir erst noch etwas zeigen.']

In [32]:
bleus = []

for i, t in enumerate(translations):
    reference = tokenizer_de.encode(de_sents_test[i]).tokens
    hypothesis = tokenizer_de.encode(t).tokens

bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  ))

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [33]:
(sum(bleus)/len(bleus))*100

22.250253290431036

## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара. 

Когда работаем с малоресурсным языком, можно сгенерировать синтетические данные. Для этого применяется backtranslation. 

Мы хотим обучить переводчик en-ru. Допустим, есть какое-то количество параллельных данных (предложений, которые есть на двух языках). Это странно, но у нас есть много данных на русском (допустим).

* Обучаем модель в обратную сторону, с ru-en (на малых данных). Получили переводчик. 

* Затем возьмем много данных на русском. Переведем их на английский язык. 

* Так у нас теперь есть дополнительный синтетический параллельный корпус; смешиваем с оригинальным корпусом и обучаем новую модель. 

! у backtranslation есть параметры. 

Как генерировать back translated data: декодер в greedy inference / beam search / сэмплирование.

Соотношение back translated данных к нормальным данным: можно увеличить выборку "нормальных" данных (добавлять несколько копий каждого предложения). 

Вывод: обратный перевод позволяет увеличить количество данных для обучения и помогает обучить модель лучше, чем если бы мы использовали только малое количество доступных данных.